In [3]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import scipy
import numpy as np

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='OJWiz15BShFHlax31OVUh-3Y0nE3kRZleXlmZ5HK-NSX',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.eu-gb.cloud-object-storage.appdomain.cloud')

def load_stream(object_key):
    # load data of type "application/octet-stream" into a botocore.response.StreamingBody object.
    # Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
    # ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
    # pandas documentation: http://pandas.pydata.org/
    bucket = 'uavteams24-donotdelete-pr-d90cszuedlklrl'

    return cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

In [7]:
bucket_name = 'uavteams24-donotdelete-pr-d90cszuedlklrl'
output_prefix = 'CSVs/'  # folder in your bucket where CSVs will be stored

# ==== FUNCTIONS ====
def load_stream(object_key):
    """Download a file object from IBM COS"""
    return cos_client.get_object(Bucket=bucket_name, Key=object_key)['Body'].read()

def read_mat_from_cos(mat_key):
    """Read a .mat file from COS into a dictionary"""
    f3_stream = load_stream(mat_key)
    with open('tmp.mat', 'wb') as f:
        f.write(f3_stream)
    data = scipy.io.loadmat('tmp.mat', appendmat=False)
    return data

def convert_and_upload(data_dict, mat_key):
    """Convert MATLAB data to CSV and upload to the CSVs folder in COS"""
    if 'QDrone_data' not in data_dict:
        print(f"⚠️ 'QDrone_data' not found in {mat_key}, skipping.")
        return

    arr = np.array(data_dict['QDrone_data'])
    df = pd.DataFrame(arr)
    base_name = os.path.splitext(os.path.basename(mat_key))[0]

    # Convert DataFrame to CSV in memory
    buffer = io.StringIO()
    df.to_csv(buffer, index=False)

    # Upload to IBM COS inside 'CSVs/' folder
    object_key = f"{output_prefix}{base_name}.csv"
    cos_client.put_object(Bucket=bucket_name, Key=object_key, Body=buffer.getvalue())

    print(f"✅ Uploaded {object_key} to bucket {bucket_name}")

def process_all_mat_files():
    """Find all .mat files in the bucket and process them"""
    response = cos_client.list_objects_v2(Bucket=bucket_name)
    if 'Contents' not in response:
        print("⚠️ No files found in the bucket.")
        return

    mat_files = [item['Key'] for item in response['Contents'] if item['Key'].endswith('.mat')]

    print(f"Found {len(mat_files)} .mat files to process.")
    for mat_key in mat_files:
        print(f"🔄 Processing {mat_key} ...")
        try:
            data = read_mat_from_cos(mat_key)
            convert_and_upload(data, mat_key)
        except Exception as e:
            print(f"❌ Error processing {mat_key}: {e}")

# ==== RUN ====
process_all_mat_files()

Found 130 .mat files to process.
🔄 Processing F0_SV0_SP1_t1_D1_R1.mat ...
✅ Uploaded CSVs/F0_SV0_SP1_t1_D1_R1.csv to bucket uavteams24-donotdelete-pr-d90cszuedlklrl
🔄 Processing F0_SV0_SP1_t1_D1_R2.mat ...
✅ Uploaded CSVs/F0_SV0_SP1_t1_D1_R2.csv to bucket uavteams24-donotdelete-pr-d90cszuedlklrl
🔄 Processing F0_SV0_SP1_t1_D1_R3.mat ...
✅ Uploaded CSVs/F0_SV0_SP1_t1_D1_R3.csv to bucket uavteams24-donotdelete-pr-d90cszuedlklrl
🔄 Processing F0_SV0_SP1_t1_D2.mat ...
✅ Uploaded CSVs/F0_SV0_SP1_t1_D2.csv to bucket uavteams24-donotdelete-pr-d90cszuedlklrl
🔄 Processing F0_SV0_SP1_t1_D3.mat ...
✅ Uploaded CSVs/F0_SV0_SP1_t1_D3.csv to bucket uavteams24-donotdelete-pr-d90cszuedlklrl
🔄 Processing F0_SV0_SP1_t2_D1_R1.mat ...
✅ Uploaded CSVs/F0_SV0_SP1_t2_D1_R1.csv to bucket uavteams24-donotdelete-pr-d90cszuedlklrl
🔄 Processing F0_SV0_SP1_t2_D1_R2.mat ...
✅ Uploaded CSVs/F0_SV0_SP1_t2_D1_R2.csv to bucket uavteams24-donotdelete-pr-d90cszuedlklrl
🔄 Processing F0_SV0_SP1_t2_D1_R3.mat ...
✅ Uploaded CSV

In [8]:
response = cos_client.list_objects_v2(Bucket='uavteams24-donotdelete-pr-d90cszuedlklrl', Prefix='CSVs/')
count = 0
for obj in response.get('Contents', []):
    count +=1
print(count)

130


In [10]:
import io
import pandas as pd

# Specify a single CSV to check
csv_key = 'CSVs/F0_SV0_SP1_t1_D1_R1.csv'  # replace with any CSV from the folder

# Download CSV from COS
csv_stream = cos_client.get_object(Bucket=bucket_name, Key=csv_key)['Body'].read()

# Load CSV into pandas DataFrame
df = pd.read_csv(io.StringIO(csv_stream.decode('utf-8')))

# Print basic info
print(f"File: {csv_key}")
print(f"Shape: {df.shape}")
print(f"First 5 rows:\n{df.head()}")
print(f"Approx. size: {len(csv_stream)/1024:.2f} KB")

response = cos_client.list_objects_v2(Bucket='uavteams24-donotdelete-pr-d90cszuedlklrl', Prefix='CSVs/')
count = 0
for obj in response.get('Contents', []):
    count +=1
print(count)

File: CSVs/F0_SV0_SP1_t1_D1_R1.csv
Shape: (56, 87837)
First 5 rows:
          0         1         2         3         4         5         6  \
0  0.000000  0.001000  0.002000  0.003000  0.004000  0.005000  0.006000   
1  0.000000 -0.000029 -0.000023 -0.000017 -0.000011 -0.000014 -0.000017   
2  0.000000  0.000041  0.000042  0.000043  0.000044  0.000058  0.000071   
3  0.000000 -0.000195 -0.000103 -0.000011  0.000082  0.000101  0.000121   
4 -0.025966 -0.025966 -0.025966 -0.025458 -0.024563 -0.023383 -0.022136   

          7         8         9  ...      87827      87828      87829  \
0  0.007000  0.008000  0.009000  ...  87.827000  87.828000  87.829000   
1 -0.000024 -0.000031 -0.000038  ...  -0.021012  -0.021027  -0.021051   
2  0.000088  0.000105  0.000122  ...   0.009294   0.009316   0.009374   
3  0.000128  0.000135  0.000142  ...   2.886666   2.886671   2.886676   
4 -0.020855 -0.019617 -0.018429  ...  -0.013405  -0.012366  -0.011387   

       87830      87831      87832      87